In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jeffheaton/glasses-or-no-glasses")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/jeffheaton/glasses-or-no-glasses


In [1]:
import torch
import torch.nn as nn

In [ ]:
class Block(nn.Module):
    def __init__(self, input_channels, output_channels, kernel_size, stride, padding):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride, padding),
            nn.InstanceNorm2d(output_channels),
            nn.LeakyReLU()
        )

    def forward(self, x):
        return self.layer(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_channels = 5,output_channels= [16, 32, 64, 128, 256, 512]):
        super().__init__()
        self.fc1 = nn.Sequential(
            nn.Conv2d(input_channels, output_channels[0], kernel_size = 4, stride = 2, padding = 1),
            nn.LeakyReLU()
        )
        layer = [Block(output_channels[i], output_channels[i+1], 4, 2, 1) for i in range(5)]

        self.backbone = nn.Sequential(*layer)

        self.fc2 = nn.Conv2d(output_channels[-1], 1, kernel_size = 4, stride = 2, padding = 0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.backbone(x)

        return self.fc2(x)

In [4]:
D_model = Discriminator()
input = torch.randn(6,5,256,256)
print(D_model(input).shape)

torch.Size([6, 1, 1, 1])


In [13]:
class GeneratorBlock(nn.Module):
    def __init__(self, input_channels, output_channels, k, s, p):
        super().__init__()
        self.layer = nn.Sequential(
            nn.ConvTranspose2d(input_channels, output_channels, kernel_size = k , stride = s, padding = p),
            nn.BatchNorm2d(output_channels),
            nn.ReLU()
        )

    def forward(self,x):
        return self.layer(x)

In [14]:
class Generator(nn.Module):
    def __init__(self, input_channels = 100 + 2, output_channels = [512, 256, 128, 64, 32, 16]):
        super().__init__()
        layer = [GeneratorBlock(output_channels[i], output_channels[i+1], 4, 2, 1) for i in range(5)]
        self.backbone = nn.Sequential(nn.ConvTranspose2d(input_channels, output_channels[0], kernel_size = 4, stride = 2, padding = 0),
                                      nn.BatchNorm2d(output_channels[0]),
                                      nn.ReLU(),
            *layer,
            nn.ConvTranspose2d(output_channels[-1], 3, kernel_size = 4, stride = 2, padding = 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.backbone(x)

In [15]:
G_model = Generator()
print(G_model)

Generator(
  (backbone): Sequential(
    (0): ConvTranspose2d(102, 512, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): GeneratorBlock(
      (layer): Sequential(
        (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (4): GeneratorBlock(
      (layer): Sequential(
        (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (5): GeneratorBlock(
      (layer): Sequential(
        (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): 

In [16]:
input = torch.randn(6,102, 1, 1)
print(G_model(input).shape)

torch.Size([6, 3, 256, 256])


In [1]:
device = torch.device('cuda' if torch.device.is_available() else 'cpu')

NameError: name 'torch' is not defined

In [ ]:
from torch.utils.data import DataLoader, ImageFolder

In [ ]:
dataset = ImageFolder(root_dir)

In [ ]:
new_data = []
for idx, (image, label) in enumerate(dataset):
    onehot = torch.zeros((1,2))
    onehot[label] = 1
    image_labels = torch.zeros(2, img_size, img_size)
    image_labels[label,:,:] = 1
    real_image_with_label = torch.cat([image_labels, image], dim = 0)

    new_data.append(image, label, onehot, real_image_with_label)


In [ ]:
data_loader = DataLoader(new_data, batch_size = 16, shuffle = True, pin_memory = True)

In [ ]:
class Model():
    def __init__(self, alpha, beta):
        self.D_model = Discriminator().to(device)
        self.G_model = Generator().to(device)
        self.D_opt = torch.optim.Adam(self.D_model.parameters(), lr = alpha)
        self.G_opt = torch.optim.Adam(self.G_model.parameters(), lr = beta)

    def GP(self, real_image, fake_image):
        batch_size,_,_,_ = real_image.shape
        alpha = torch.randn(batch_size,1,1,1)
        interpolated_image = alpha * real_image + (1 - alpha) * fake_image
        interpolated_image.requires_grad = True
        output = self.D_model(interpolated_image)
        grad = torch.autograd.grad(
            outputs = output,
            inputs = interpolated_image,
            grad_outputs = torch.ones_like(interpolated_image)
        )
        grad = grad.reshape(batch_size, -1)
        grad_norm = grad.norm(2, dim = 1)
        penalty = torch.mean((grad_norm - 1)**2)

        return penalty
    
    def train(self):
        for idx , (image, label, onehot, real_image_with_label) in enumerate(data_loader):
            batch_size,_,img_size , _ = image.shape
            real_image_with_label = real_image_with_label.to(device)
            image = image.to(device)
            label = label.to(device)
            onehot = onehot.to(device)

            noise = torch.randn(batch_size, 100, 1, 1)
            one_hot = one_hot.reshape(batch_size, 2, 1, 1)
            noise = torch.cat([noise, one_hot], dim = 1)
            noise = noise.to(device)            
            fake_image = self.G_model(noise)


            fake_label = real_image_with_label[:, 3:, :, :]
            fake_label = fake_label.to(device)
            
            fake_image_with_label = torch.cat([fake_image, fake_label], dim = 1)

            D_real_pred = self.D_model(real_image_with_label)
            D_fake_pred = self.D_model(fake_image_with_label.detach())

            D_loss = D_fake_pred - D_real_pred + self.lembda * self.GP(fake_image, image.to(device))

            self.D_opt.zero_grad()

            D_loss.backward()

            self.D_opt.step()

            G_loss = -torch.mean(self.D_model(fake_image_with_label))

            self.G_opt.zero_grad()

            G_loss.backward()

            self.G_opt.step()

In [1]:
import torch

In [9]:
a = torch.rand(1,2)
print(a)

tensor([[0.7008, 0.3281]])


In [10]:
a = a.reshape(2,1,1)
print(a)

tensor([[[0.7008]],

        [[0.3281]]])


In [3]:
a = a.unsqueeze(2)

In [6]:
a = a.transpose(0,1)

In [7]:
print(a)

tensor([[[0.4102]],

        [[0.1985]]])


In [8]:
print(a.shape)

torch.Size([2, 1, 1])
